In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact, Select, Dropdown
from IPython.display import display
from IPython.display import Image

![chapter_network.png](chapter_network.png)

In [ ]:
df= pd.read_csv('datasets\dataset.csv')
df=df.sort_values(by=['user_id','date_created'])
df['source'] = df['book'] + "_" + df['chapter'].str.strip()
df['target'] = df['source'].shift(-1)
df['from'] = df['source'].shift(1)
df['user_next'] = df['user_id'].shift(-1)
df['user_prev'] = df['user_id'].shift(1)
df.loc[df['user_id'] != df['user_next'], 'target'] = 'last'
df.loc[df['user_id'] != df['user_prev'], 'from'] = 'first'

df['same'] = df['source'] == df['target']
df['pair'] = df['user_id'].astype(str) + "#" + df['source'] + "#" + df['target']
df.drop_duplicates(subset=['pair'], keep='first', inplace=True)
df.drop(labels=['pair', 'user_next', 'user_prev'], axis='columns', inplace=True)
df = df[df['target'] != 'last']
G=nx.from_pandas_edgelist(df, 'source', 'target', create_using=nx.DiGraph)
out=nx.out_degree_centrality(G)
nx.set_node_attributes(G, out, 'out-degree')
bb = nx.betweenness_centrality(G)
nx.set_node_attributes(G, bb, 'betweenness')
eigen= nx.eigenvector_centrality(G)
nx.set_node_attributes(G, eigen, 'eigen')
data_list=list(G.nodes(data=True))
data={}
data['chapter']=[x[0] for x in data_list]
data['out-degree'] = [x[1]['out-degree'] for x in data_list]
data['eigen'] = [x[1]['eigen'] for x in data_list]
data['betweenness'] = [x[1]['betweenness'] for x in data_list]
df1 = pd.DataFrame(data)

In [ ]:
def view(book='', chapter=''):
    if chapter==None:
        return None
    source = book + "_" + chapter.strip()
    row1=[n for n in G[source]]
    df2= pd.DataFrame(row1)
    df3=df2.set_index(0).join(df1.set_index('chapter'))
    df3=df3.sort_values(by=['eigen'],ascending=False)
    df3=df3.rename_axis('book')
    return df3

books=sorted(df.book.unique().tolist())
w = Select(options=books)
c = Select()

def update(*args):
    chapters = sorted(df[df.book==w.value].chapter.str.strip().unique().tolist())
    c.options = chapters

w.observe(update)

v = interact(view, book=w, chapter=c)
display(v);

In [ ]:
# using the Voila button when uncommenting the next cell causes an endless loop   
# to end it you need to turn off conda

In [ ]:
#!voila widget.ipynb --enable_nbextensions=True